This script creates a dataset of images and label files 

In [1]:
import random
from PIL import Image, ImageOps, ImageEnhance
import os
import uuid


In [2]:
def list_images(directory):
    """ List all images in the specified directory. """
    return [file for file in os.listdir(directory) if file.endswith(('.png', '.jpg', '.jpeg'))]

def check_overlap(new_bbox, existing_bboxes, overlap_threshold=0.1):
    """Check if the new bounding box overlaps significantly with existing ones."""
    new_x1, new_y1, new_x2, new_y2 = new_bbox
    for bbox in existing_bboxes:
        x1, y1, x2, y2 = bbox
        # Calculate overlap area
        overlap_x1 = max(new_x1, x1)
        overlap_y1 = max(new_y1, y1)
        overlap_x2 = min(new_x2, x2)
        overlap_y2 = min(new_y2, y2)
        overlap_area = max(0, overlap_x2 - overlap_x1) * max(0, overlap_y2 - overlap_y1)
        new_area = (new_x2 - new_x1) * (new_y2 - new_y1)
        # Check if the overlap is significant
        if overlap_area / new_area > overlap_threshold:
            return True
    return False

def place_assets_on_map(map_image_path, asset_image_paths, output_path, max_assets=5, max_size=(120, 120), transparency_chance=0.1):
    """
    Places multiple asset images on a map image at random locations and saves the result.
    Args:
    - map_image_path (str): Path to the map image.
    - asset_image_paths (list): Paths to the asset images.
    - output_path (str): Path to save the combined image.
    - max_assets (int): Maximum number of assets to place.
    Returns:
    - bboxes (list): List of bounding boxes for each asset.
    """
    # Load the base (map) image
    map_image = Image.open(map_image_path)
    placed_bboxes = []

    for _ in range(random.randint(1, max_assets)):
        asset_image_path = random.choice(asset_image_paths)
        asset_image = Image.open(asset_image_path).convert("RGBA")

        # Resize the asset image
        asset_image.thumbnail(max_size, Image.ANTIALIAS)

        # Random horizontal mirroring
        if random.choice([True, False]):
            asset_image = ImageOps.mirror(asset_image)

        # Apply transparency randomly
        if random.random() < transparency_chance:
            alpha = asset_image.split()[3]
            alpha = ImageEnhance.Brightness(alpha).enhance(0.5)  # Adjust transparency level
            asset_image.putalpha(alpha)

        for _ in range(100):  # Attempt up to 100 times to find a non-overlapping position
            x = random.randint(0, map_image.width - asset_image.width)
            y = random.randint(0, map_image.height - asset_image.height)
            bbox = (x, y, x + asset_image.width, y + asset_image.height)

            if not check_overlap(bbox, placed_bboxes):
                map_image.paste(asset_image, (x, y), asset_image)
                placed_bboxes.append(bbox)
                break

    map_image.save(output_path)
    return placed_bboxes

def get_label_from_filename(filename):
    """ Extracts the label from the filename. """
    return filename.split('_')[0]

def group_assets_by_label(assets):
    """ Groups asset filenames by their label. """
    grouped_assets = {}
    for asset in assets:
        label = get_label_from_filename(asset)
        if label in grouped_assets:
            grouped_assets[label].append(asset)
        else:
            grouped_assets[label] = [asset]
    return grouped_assets

def normalize_bbox(bbox, width, height):
    """ Normalizes bounding box coordinates. """
    x_center = (bbox[0] + bbox[2]) / 2 / width
    y_center = (bbox[1] + bbox[3]) / 2 / height
    bbox_width = (bbox[2] - bbox[0]) / width
    bbox_height = (bbox[3] - bbox[1]) / height
    return x_center, y_center, bbox_width, bbox_height

In [3]:
# Number of images to create per label
num_images_per_label = 30

# Directories
map_assets_dir = 'map_assets'
assets_dir = 'assets'

# Listing and grouping images
map_assets = list_images(map_assets_dir)
assets = list_images(assets_dir)
grouped_assets = group_assets_by_label(assets)

# Main loop for generating images
for label, asset_filenames in grouped_assets.items():
    for _ in range(num_images_per_label):
        unique_id = uuid.uuid4() 

        base_image_name = random.choice(map_assets)
        map_image_path = os.path.join(map_assets_dir, base_image_name)

        # Ensure a diverse selection of assets
        num_different_assets = random.randint(1, len(asset_filenames))
        selected_asset_filenames = random.sample(asset_filenames, num_different_assets)

        asset_image_paths = [os.path.join(assets_dir, name) for name in selected_asset_filenames]

        output_image_path = f'images/{label}_{unique_id}.jpg'
        labels_file_path = f'labels/{label}_{unique_id}.txt'

        bboxes = place_assets_on_map(map_image_path, asset_image_paths, output_image_path)

        with open(labels_file_path, 'w') as file:
            for bbox in bboxes:
                # Normalize and write bounding box
                width, height = Image.open(map_image_path).size
                x_center, y_center, bbox_width, bbox_height = normalize_bbox(bbox, width, height)
                file.write(f"{label} {x_center} {y_center} {bbox_width} {bbox_height}\n")

C:\Users\Grego\AppData\Local\Temp\ipykernel_10372\2669640881.py:42: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  asset_image.thumbnail(max_size, Image.ANTIALIAS)
